In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.Timestamp.now(tz="UTC").floor('H')
print(f'{current_date=}')

current_date=Timestamp('2025-03-30 05:00:00+0000', tz='UTC')


In [4]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

2025-03-30 00:56:53,706 INFO: Initializing external client
2025-03-30 00:56:53,707 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-30 00:56:55,039 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1220769
Fetching data from 2025-03-02 05:00:00+00:00 to 2025-03-30 04:00:00+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.32s) 
ts_data:                     pickup_hour  rides  pickup_longitude  pickup_latitude  \
57608 2025-03-02 05:00:00+00:00      1        -87.721559        41.968069   
47169 2025-03-02 06:00:00+00:00      0        -87.721559        41.968069   
17372 2025-03-02 07:00:00+00:00      1        -87.721559        41.968069   
42067 2025-03-02 08:00:00+00:00      0        -87.721559        41.968069   
41886 2025-03-02 09:00:00+00:00      1        -87.721559        41.968069   
...                         ...    ...               ...              ...   
45476 2025-03-30 00:00:00

In [5]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

2025-03-30 00:57:03,288 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-30 00:57:03,293 INFO: Initializing external client
2025-03-30 00:57:03,293 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-30 00:57:04,418 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1220769


In [6]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location,predicted_demand,pickup_hour
0,Albany Park,1.0,2025-03-30 05:00:00+00:00
1,Andersonville,0.0,2025-03-30 05:00:00+00:00
2,Archer Heights,0.0,2025-03-30 05:00:00+00:00
3,Armour Square,0.0,2025-03-30 05:00:00+00:00
4,Ashburn,0.0,2025-03-30 05:00:00+00:00
...,...,...,...
90,West Ridge,1.0,2025-03-30 05:00:00+00:00
91,West Town,0.0,2025-03-30 05:00:00+00:00
92,Wicker Park,0.0,2025-03-30 05:00:00+00:00
93,Woodlawn,0.0,2025-03-30 05:00:00+00:00


In [7]:
from src.feature_store_api import get_feature_store
import src.config as config

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key= ['pickup_location', 'pickup_hour'],
    event_time='pickup_hour',
)

2025-03-30 00:57:08,199 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-30 00:57:08,201 INFO: Initializing external client
2025-03-30 00:57:08,201 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-30 00:57:09,378 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1220769


In [8]:
feature_group.insert(predictions, write_options={'wait_for_job': False})

Uploading Dataframe: 100.00% |██████████| Rows 95/95 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1220769/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(Job('model_predictions_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)